In [ ]:
!pip install pycaret==2.3.10 markupsafe==2.0.1 pyyaml==5.4.1 -qq

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

# 시각화 라이브러리
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 파일 불러오기
train = pd.read_csv("/content/drive/MyDrive/2022-01/빅데이터마이닝/tasks/archive/train.csv")
test = pd.read_csv("/content/drive/MyDrive/2022-01/빅데이터마이닝/tasks/archive/test.csv")

submission = pd.read_csv("/content/drive/MyDrive/2022-01/빅데이터마이닝/tasks/archive/sample_submission.csv")
meal = pd.read_csv("/content/drive/MyDrive/2022-01/빅데이터마이닝/tasks/archive/meal_info.csv")
center = pd.read_csv("/content/drive/MyDrive/2022-01/빅데이터마이닝/tasks/archive/fulfilment_center_info.csv")

# 데이터 살펴보기

In [ ]:
print("train : ",train.shape)
print("test : ",test.shape)
print("meal : ",meal.shape)
print("center : ",center.shape)
print("submission : ",submission.shape)

train :  (456548, 9)
test :  (32573, 8)
meal :  (51, 3)
center :  (77, 5)
submission :  (32573, 2)


In [ ]:
print("train : ",train.isnull().sum())
print("===================")
print("test : ",test.isnull().sum())
print("===================")
print("meal : ",meal.isnull().sum())
print("===================")
print("center : ",center.isnull().sum())
print("===================")
print("submission : ",submission.isnull().sum())

train :  id                       0
week                     0
center_id                0
meal_id                  0
checkout_price           0
base_price               0
emailer_for_promotion    0
homepage_featured        0
num_orders               0
dtype: int64
test :  id                       0
week                     0
center_id                0
meal_id                  0
checkout_price           0
base_price               0
emailer_for_promotion    0
homepage_featured        0
dtype: int64
meal :  meal_id     0
category    0
cuisine     0
dtype: int64
center :  center_id      0
city_code      0
region_code    0
center_type    0
op_area        0
dtype: int64
submission :  id            0
num_orders    0
dtype: int64


# train

In [ ]:
# 데이터 살펴보기
train.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
0,1379560,1,55,1885,136.83,152.29,0,0,177
1,1466964,1,55,1993,136.83,135.83,0,0,270
2,1346989,1,55,2539,134.86,135.86,0,0,189
3,1338232,1,55,2139,339.50,437.53,0,0,54
4,1448490,1,55,2631,243.50,242.50,0,0,40


In [ ]:
train.tail()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
456543,1271326,145,61,1543,484.09,484.09,0,0,68
456544,1062036,145,61,2304,482.09,482.09,0,0,42
456545,1110849,145,61,2664,237.68,321.07,0,0,501
456546,1147725,145,61,2569,243.50,313.34,0,0,729
456547,1361984,145,61,2490,292.03,290.03,0,0,162


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456548 entries, 0 to 456547
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   id                     456548 non-null  int64  
 1   week                   456548 non-null  int64  
 2   center_id              456548 non-null  int64  
 3   meal_id                456548 non-null  int64  
 4   checkout_price         456548 non-null  float64
 5   base_price             456548 non-null  float64
 6   emailer_for_promotion  456548 non-null  int64  
 7   homepage_featured      456548 non-null  int64  
 8   num_orders             456548 non-null  int64  
dtypes: float64(2), int64(7)
memory usage: 31.3 MB


In [ ]:
train.columns

Index(['id', 'week', 'center_id', 'meal_id', 'checkout_price', 'base_price',
       'emailer_for_promotion', 'homepage_featured', 'num_orders'],
      dtype='object')

In [ ]:
train = train.astype({'id': 'object', 'center_id' : 'object', 'meal_id' : 'object'})
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456548 entries, 0 to 456547
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   id                     456548 non-null  object 
 1   week                   456548 non-null  int64  
 2   center_id              456548 non-null  object 
 3   meal_id                456548 non-null  object 
 4   checkout_price         456548 non-null  float64
 5   base_price             456548 non-null  float64
 6   emailer_for_promotion  456548 non-null  int64  
 7   homepage_featured      456548 non-null  int64  
 8   num_orders             456548 non-null  int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 31.3+ MB


# Feature Engineering

In [ ]:
### 더미변수화
train_dum = pd.get_dummies(train, columns = ['emailer_for_promotion', 'homepage_featured'], drop_first=False)
train_dum

,id,week,center_id,meal_id,checkout_price,base_price,num_orders,emailer_for_promotion_0,emailer_for_promotion_1,homepage_featured_0,homepage_featured_1
0,1379560,1,55,1885,136.83,152.29,177,1,0,1,0
1,1466964,1,55,1993,136.83,135.83,270,1,0,1,0
2,1346989,1,55,2539,134.86,135.86,189,1,0,1,0
3,1338232,1,55,2139,339.50,437.53,54,1,0,1,0
4,1448490,1,55,2631,243.50,242.50,40,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
456543,1271326,145,61,1543,484.09,484.09,68,1,0,1,0
456544,1062036,145,61,2304,482.09,482.09,42,1,0,1,0
456545,1110849,145,61,2664,237.68,321.07,501,1,0,1,0
456546,1147725,145,61,2569,243.50,313.34,729,1,0,1,0


## pycaret 사용하기

In [ ]:
from pycaret.regression import *

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [ ]:
## dummies X
s = setup(train, target = 'num_orders')

,Description,Value
0,session_id,412
1,Target,num_orders
2,Original Data,"(456548, 9)"
3,Missing Values,False
4,Numeric Features,6
5,Categorical Features,2
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(319583, 8)"


In [ ]:
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,80.6157,36595.3034,191.0897,0.7677,0.5426,0.5749,165.540
et,Extra Trees Regressor,85.4372,38457.5456,195.7899,0.7562,0.5709,0.6233,80.152
lightgbm,Light Gradient Boosting Machine,108.7228,41693.5905,204.0719,0.7350,0.7605,0.9741,1.885
gbr,Gradient Boosting Regressor,138.6750,69942.1452,264.2473,0.5559,0.9068,1.3942,42.941
dt,Decision Tree Regressor,107.8228,72577.7336,268.9324,0.5374,0.7058,0.6758,2.586
lr,Linear Regression,203.6257,126761.1031,355.8228,0.1947,1.2873,2.4341,0.411
ridge,Ridge Regression,203.6255,126761.1047,355.8228,0.1947,1.2873,2.4341,0.061
lar,Least Angle Regression,203.6257,126761.1046,355.8228,0.1947,1.2873,2.4341,0.069
br,Bayesian Ridge,203.6245,126761.1048,355.8227,0.1947,1.2873,2.4341,0.147
lasso,Lasso Regression,203.4419,126786.5915,355.8571,0.1945,1.2851,2.4415,0.503


In [ ]:
evaluate_model(best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [ ]:
predictions = predict_model(best, data = train)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,44.7086,13309.1461,115.3653,0.9151,0.364,0.3167


In [ ]:
save_model(best, 'FE_NONE')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[],
                                       target='num_orders', time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strat...
                  RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                        criterion='mse', max_depth=None,
                                        max_features='auto', max_leaf_nodes=None,
                                        max_samples=None,
                             

In [ ]:
## Feature Engineering O
a = setup(train_dum, target = 'num_orders')

,Description,Value
0,session_id,2388
1,Target,num_orders
2,Original Data,"(456548, 11)"
3,Missing Values,False
4,Numeric Features,10
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(319583, 8)"


In [ ]:
a_best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,80.6506,36321.0111,190.4283,0.7692,0.5433,0.5758,143.654
et,Extra Trees Regressor,85.6440,38413.1575,195.8314,0.7559,0.5733,0.6288,72.021
lightgbm,Light Gradient Boosting Machine,108.8089,41886.3833,204.5760,0.7336,0.7630,0.9756,1.795
dt,Decision Tree Regressor,106.9323,66884.9465,258.4421,0.5737,0.7071,0.6739,3.386
gbr,Gradient Boosting Regressor,138.5417,69822.0322,264.1205,0.5561,0.9083,1.3992,37.778
lr,Linear Regression,203.6734,126655.5851,355.8143,0.1945,1.2890,2.4379,0.607
ridge,Ridge Regression,203.6732,126655.5854,355.8143,0.1945,1.2890,2.4379,0.080
lar,Least Angle Regression,203.6734,126655.5864,355.8143,0.1945,1.2890,2.4379,0.093
br,Bayesian Ridge,203.6722,126655.5870,355.8143,0.1945,1.2889,2.4379,0.200
lasso,Lasso Regression,203.4890,126681.0875,355.8500,0.1943,1.2865,2.4454,0.489


In [ ]:
evaluate_model(a_best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [ ]:
a_predictions = predict_model(a_best, data = train_dum)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,44.6972,13653.4609,116.848,0.9129,0.3633,0.3157


In [ ]:
save_model(a_best, 'FE_DONE')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[],
                                       target='num_orders', time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strat...
                  RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                        criterion='mse', max_depth=None,
                                        max_features='auto', max_leaf_nodes=None,
                                        max_samples=None,
                             

## train_df로 pycaret 시도

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/2022-01/빅데이터마이닝/tasks/archive/train_df.csv")

In [ ]:
train_df

,id,week,center_id,city_code,region_code,center_type,op_area,meal_id,category,cuisine,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
0,1379560,1,55,647,56,TYPE_C,2.0,1885,Beverages,Thai,136.83,152.29,0,0,177
1,1466964,1,55,647,56,TYPE_C,2.0,1993,Beverages,Thai,136.83,135.83,0,0,270
2,1346989,1,55,647,56,TYPE_C,2.0,2539,Beverages,Thai,134.86,135.86,0,0,189
3,1338232,1,55,647,56,TYPE_C,2.0,2139,Beverages,Indian,339.50,437.53,0,0,54
4,1448490,1,55,647,56,TYPE_C,2.0,2631,Beverages,Indian,243.50,242.50,0,0,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456543,1271326,145,61,473,77,TYPE_A,4.5,1543,Desert,Indian,484.09,484.09,0,0,68
456544,1062036,145,61,473,77,TYPE_A,4.5,2304,Desert,Indian,482.09,482.09,0,0,42
456545,1110849,145,61,473,77,TYPE_A,4.5,2664,Salad,Italian,237.68,321.07,0,0,501
456546,1147725,145,61,473,77,TYPE_A,4.5,2569,Salad,Italian,243.50,313.34,0,0,729


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456548 entries, 0 to 456547
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   id                     456548 non-null  int64  
 1   week                   456548 non-null  int64  
 2   center_id              456548 non-null  int64  
 3   city_code              456548 non-null  int64  
 4   region_code            456548 non-null  int64  
 5   center_type            456548 non-null  object 
 6   op_area                456548 non-null  float64
 7   meal_id                456548 non-null  int64  
 8   category               456548 non-null  object 
 9   cuisine                456548 non-null  object 
 10  checkout_price         456548 non-null  float64
 11  base_price             456548 non-null  float64
 12  emailer_for_promotion  456548 non-null  int64  
 13  homepage_featured      456548 non-null  int64  
 14  num_orders             456548 non-nu

In [ ]:
## 더미변수화
train_df_dum = pd.get_dummies(train_df, columns = ['emailer_for_promotion', 'homepage_featured'], drop_first=False)
train_df_dum

,id,week,center_id,city_code,region_code,center_type,op_area,meal_id,category,cuisine,checkout_price,base_price,num_orders,emailer_for_promotion_0,emailer_for_promotion_1,homepage_featured_0,homepage_featured_1
0,1379560,1,55,647,56,TYPE_C,2.0,1885,Beverages,Thai,136.83,152.29,177,1,0,1,0
1,1466964,1,55,647,56,TYPE_C,2.0,1993,Beverages,Thai,136.83,135.83,270,1,0,1,0
2,1346989,1,55,647,56,TYPE_C,2.0,2539,Beverages,Thai,134.86,135.86,189,1,0,1,0
3,1338232,1,55,647,56,TYPE_C,2.0,2139,Beverages,Indian,339.50,437.53,54,1,0,1,0
4,1448490,1,55,647,56,TYPE_C,2.0,2631,Beverages,Indian,243.50,242.50,40,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456543,1271326,145,61,473,77,TYPE_A,4.5,1543,Desert,Indian,484.09,484.09,68,1,0,1,0
456544,1062036,145,61,473,77,TYPE_A,4.5,2304,Desert,Indian,482.09,482.09,42,1,0,1,0
456545,1110849,145,61,473,77,TYPE_A,4.5,2664,Salad,Italian,237.68,321.07,501,1,0,1,0
456546,1147725,145,61,473,77,TYPE_A,4.5,2569,Salad,Italian,243.50,313.34,729,1,0,1,0


In [ ]:
# 그냥 df
df1 = setup(train_df, target = 'num_orders')

,Description,Value
0,session_id,2120
1,Target,num_orders
2,Original Data,"(456548, 15)"
3,Missing Values,False
4,Numeric Features,8
5,Categorical Features,6
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(319583, 39)"


In [ ]:
df1_best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,9.182280e+01,3.057573e+04,1.747788e+02,8.088000e-01,0.6657,0.7475,2.557
dt,Decision Tree Regressor,9.580250e+01,4.874233e+04,2.201493e+02,6.943000e-01,0.6611,0.6107,3.905
gbr,Gradient Boosting Regressor,1.170350e+02,5.342317e+04,2.309734e+02,6.664000e-01,0.8137,0.9764,70.917
ridge,Ridge Regression,1.635077e+02,9.305622e+04,3.048787e+02,4.188000e-01,1.0460,1.6274,0.120
br,Bayesian Ridge,1.635006e+02,9.305622e+04,3.048787e+02,4.188000e-01,1.0459,1.6273,1.008
lr,Linear Regression,1.639710e+02,9.384068e+04,3.061338e+02,4.141000e-01,1.0392,1.6665,0.804
lasso,Lasso Regression,1.616400e+02,9.412899e+04,3.066307e+02,4.121000e-01,1.0285,1.5980,3.848
omp,Orthogonal Matching Pursuit,1.967254e+02,1.245558e+05,3.527822e+02,2.217000e-01,1.2548,2.2743,0.136
en,Elastic Net,1.953859e+02,1.276571e+05,3.571290e+02,2.024000e-01,1.2203,2.3752,6.574
knn,K Neighbors Regressor,2.046986e+02,1.524589e+05,3.903316e+02,4.700000e-02,1.1624,2.1837,4.485


In [ ]:
evaluate_model(df1_best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [ ]:
predictions = predict_model(df1_best, data = train_df)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,90.9055,27474.2665,165.7536,0.8247,0.6629,0.7501


In [ ]:
save_model(df1_best, 'df_FE_NONE')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[],
                                       target='num_orders', time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strat...
                  LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                colsample_bytree=1.0, importance_type='split',
                                learning_rate=0.1, max_depth=-1,
                                min_child_samples=20, min_child_weight=0.001,
                  

In [ ]:
# FE df
df2 = setup(train_df_dum, target = 'num_orders')

,Description,Value
0,session_id,7824
1,Target,num_orders
2,Original Data,"(456548, 17)"
3,Missing Values,False
4,Numeric Features,12
5,Categorical Features,4
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(319583, 39)"


In [ ]:
df2_best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,9.134410e+01,2.975680e+04,1.723502e+02,8.082000e-01,0.6627,0.7435,2.859
dt,Decision Tree Regressor,9.584780e+01,4.516150e+04,2.122605e+02,7.085000e-01,0.6619,0.6106,3.879
ridge,Ridge Regression,1.626291e+02,8.892045e+04,2.979973e+02,4.269000e-01,1.0407,1.6206,0.135
br,Bayesian Ridge,1.626224e+02,8.892046e+04,2.979973e+02,4.269000e-01,1.0406,1.6205,1.101
lasso,Lasso Regression,1.607768e+02,8.997786e+04,2.997630e+02,4.201000e-01,1.0247,1.5905,3.761
lr,Linear Regression,1.649992e+02,9.000616e+04,2.998248e+02,4.198000e-01,1.0408,1.6657,0.665
omp,Orthogonal Matching Pursuit,1.962161e+02,1.201998e+05,3.465596e+02,2.247000e-01,1.2506,2.2755,0.155
en,Elastic Net,1.948241e+02,1.230915e+05,3.506879e+02,2.062000e-01,1.2209,2.3680,5.085
knn,K Neighbors Regressor,2.036264e+02,1.462782e+05,3.823484e+02,5.610000e-02,1.1588,2.1688,6.043
llar,Lasso Least Angle Regression,2.277436e+02,1.550029e+05,3.935721e+02,-0.000000e+00,1.3839,3.3004,0.155


In [ ]:
evaluate_model(df2_best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [ ]:
predictions = predict_model(df2_best, data = train_df_dum)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,90.1427,27869.0741,166.9403,0.8222,0.6588,0.7383


In [ ]:
save_model(df2_best, 'df_FE_DONE')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[],
                                       target='num_orders', time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strat...
                  LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                colsample_bytree=1.0, importance_type='split',
                                learning_rate=0.1, max_depth=-1,
                                min_child_samples=20, min_child_weight=0.001,
                  